In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import numpy as np
from pymongo import MongoClient
from nacl.signing import SigningKey
from syft import PhiTensor
from nacl.signing import VerifyKey
from nacl.encoding import HexEncoder
from syft.core.node.common.node_table.user import NoSQLSyftUser
from syft.core.node.common.node_manager.role_manager import NewRoleManager
from syft.core.node.common.node_manager.user_manager import NoSQLUserManager
from nacl.encoding import HexEncoder

/Users/rasswanth/PySyft/.tox/syft.jupyter/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [ ]:
role = NewRoleManager()

In [ ]:

client = MongoClient(host='localhost', port=56287, username="root", password="example", uuidRepresentation="standard")

In [ ]:
if client.server_info():
    print("Valid DB")

In [ ]:
db = client["app"]

In [ ]:
db.list_collection_names()

In [ ]:
collection = db["syft_users"]


In [ ]:
user_manager = NoSQLUserManager(db)

In [ ]:
signing_key = SigningKey.generate()
verify_key = signing_key.verify_key

In [ ]:
_node_private_key = signing_key.encode(encoder=HexEncoder).decode("utf-8")  # type: ignore
_verify_key = signing_key.verify_key.encode(encoder=HexEncoder).decode("utf-8")

In [ ]:
user_manager.signup(name="mystogen",email="mystogen@fairy.tail",password="erza",role = role.owner_role,budget=5.5,private_key=_node_private_key,verify_key=_verify_key)

In [ ]:
zerek = user_manager.find_one({"email": "mystogen@fairy.tail"})

In [ ]:
zerek

In [ ]:
user_manager.drop()

In [ ]:
for user in user_manager.all():
    print(user)
    print("\n"*5)

In [ ]:
# to delete all documents in the table.
# user_manager.drop()

In [ ]:
verify_key = VerifyKey("725d5435741e75258ceaa07666f108f1adad085c640617633b74770c43fb83d0",encoder=HexEncoder)

In [ ]:
verify_key

In [ ]:
from syft.core.node.common.node_table.utils import syft_object_to_json

In [ ]:
json  = syft_object_to_json(user_manager.get_user(verify_key))

In [ ]:
json["role"] = json["role"].get("name",None)

In [ ]:
del json["private_key"]

In [ ]:
user_manager.get_budget_for_user(verify_key=verify_key)

In [ ]:
user_manager.get_user(verify_key)

In [ ]:
user_manager.can_triage_requests(verify_key)

In [ ]:
user_manager.get_user(verify_key).role

In [ ]:
user_manager.first(id_int=1)

In [ ]:
###User Login Test

In [2]:
canada = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


In [3]:
canada.users

,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,id_int,budget_spent
0,info@openmined.org,Jane Doe,5.55,e8ddb58fce9f2015d41a09541589941ec3038961488955...,Owner,,,,None,2022-09-02 03:34:29.426432,1,5.55
1,info@openmined.org,Jane Doe,5.55,0a6ddeea9e09f60334f77282be6bad54b57b3e0763a1cf...,Owner,,,,None,2022-09-02 03:34:29.517535,2,5.55
2,info@openmined.org,Jane Doe,5.55,c213e98c0a5db2d0d623ec497420a214f5c0f52d592296...,Owner,,,,None,2022-09-02 03:34:29.617618,3,5.55


In [5]:
canada.users.create(**{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget": 1000,
    }
)

User created successfully!


In [10]:
canada.users

,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,id_int,budget_spent
0,info@openmined.org,Jane Doe,5.55,e8ddb58fce9f2015d41a09541589941ec3038961488955...,Owner,,,,None,2022-09-02 03:34:29.426432,1,5.55
1,info@openmined.org,Jane Doe,5.55,0a6ddeea9e09f60334f77282be6bad54b57b3e0763a1cf...,Owner,,,,None,2022-09-02 03:34:29.517535,2,5.55
2,info@openmined.org,Jane Doe,5.55,c213e98c0a5db2d0d623ec497420a214f5c0f52d592296...,Owner,,,,None,2022-09-02 03:34:29.617618,3,5.55
3,sheldon@caltech.edu,Sheldon Cooper,1000.00,25b4cdda0425fe85aa27cc9e3cad12c79e741462bd1ebd...,Data Scientist,Jane Doe,,,,2022-09-02 03:35:36.833685,4,1000.00


In [ ]:
data = sy.Tensor(np.array([1,2,3]))

In [ ]:
phi_tensor = data.private(min_val=0,max_val=3,data_subjects=np.array(["zerek"]*3))

In [ ]:
phi_tensor

In [ ]:
phi_ptr = phi_tensor.send(canada)

In [ ]:
pub_res = phi_ptr.publish(sigma=1e5)

In [ ]:
pub_res.get_copy()

In [ ]:
canada.users

In [ ]:
canada.users

In [ ]:
from syft.core.node.common.node_service.user_manager.user_messages import (
    UpdateUserMessage,
)

In [ ]:
# Upgrade admins budget
content = {"user_id": 1, "budget": 500}
canada._perform_grid_request(grid_msg=UpdateUserMessage, content=content)

In [ ]:
canada.users